In [2]:
import os, sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)
# --- 
from techminer2 import *
import pandas as pd
directory = "mateo"

tm2__import_scopus_files(directory)
# import_references()
# annual_scientific_production()
# main_information(directory)

# with open(os.path.join(directory, "processed/_documents.csv"), "r") as f:
#     documents = pd.read_csv(f)
#     abstracts = documents["abstract"]
#     abstracts = abstracts.dropna()
#     abstracts = abstracts.str.split(".")
#     abstracts = abstracts.map(lambda x: [y for y in x if y != ""])
#     abstracts = abstracts.str[-1]
# abstracts[0]

--INFO-- Concatenating raw files in mateo/raw/cited_by/
--INFO-- Concatenating raw files in mateo/raw/references/
--INFO-- Concatenating raw files in mateo/raw/documents/
--INFO-- Applying scopus tags to database files
--INFO-- Formating column names in database files
--INFO-- Dropping NA columns in database files
--INFO-- Removing accents in database files
--INFO-- Removing stranger chars in database files
--INFO-- Processing `abstract` column
--INFO-- Processing `authors_id` column
--INFO-- Processing `title` column
--INFO-- Processing `document_type` column
--INFO-- Processing `doi` column
--INFO-- Processing `eissn` column
--INFO-- Processing `global_citations` column
--INFO-- Processing `isbn` column
--INFO-- Processing `issn` column
--INFO-- Processing `raw_authors` column
--INFO-- Processing `source_abbr` column
--INFO-- Processing `source_name` column
--INFO-- Processing `global_references` column
--INFO-- Creating `authors` column


KeyboardInterrupt: 

In [ ]:
import pandas as pd

directory = "regtech"
pd.read_csv(f"{directory}/processed/_documents.csv").local_references

In [ ]:
import glob
import sys

def _extract_keywords_from_database_files(directory):
    keywords_list = []
    files = list(glob.glob(os.path.join(directory, "processed/_*.csv")))
    for file in files:
        data = pd.read_csv(file, encoding="utf-8")
        if "raw_index_keywords" in data.columns:
            keywords_list += data.raw_index_keywords.dropna().tolist()
        if "raw_author_keywords" in data.columns:
            keywords_list += data.raw_author_keywords.dropna().tolist()
    keywords_list = pd.DataFrame({"keyword": keywords_list})
    keywords_list = keywords_list.assign(keyword=keywords_list.keyword.str.split(";"))
    keywords_list = keywords_list.explode("keyword")
    keywords_list = keywords_list.keyword.str.strip()
    keywords_list = keywords_list.drop_duplicates()
    keywords_list = keywords_list.reset_index(drop=True)
    return keywords_list


In [ ]:
from nltk.stem import PorterStemmer
from textblob import TextBlob
import pandas as pd

def _extract_keywords_roots_from_database_files(directory):
    
    keywords_list = []
    
    files = list(glob.glob(os.path.join(directory, "processed/_*.csv")))
    for file in files:
        data = pd.read_csv(file, encoding="utf-8")
        for column in ["raw_author_keywords", "raw_index_keywords"]:
            keywords_list.append(data[column])

    keywords_list = pd.concat(keywords_list)
    keywords_list = keywords_list.dropna()
    keywords_list = keywords_list.str.split(";")
    keywords_list = keywords_list.explode()
    keywords_list = keywords_list.str.strip()
    keywords_list = keywords_list.drop_duplicates()

    keywords_list = keywords_list.str.replace(r"\[.+\]", "", regex=True)
    keywords_list = keywords_list.str.replace(r"\(.+\)", "", regex=True)
    keywords_list = keywords_list.str.replace(r"-", " ", regex=False)
    keywords_list = keywords_list.str.replace(r"&", " ", regex=False)

    # list of single words
    keywords_list = keywords_list.str.split()
    s = PorterStemmer()
    keywords_list = keywords_list.map(lambda x: [s.stem(y) for y in x])
    keywords_list = keywords_list.map(set)
    keywords_list = keywords_list.map(sorted)
    keywords_list = keywords_list.map(lambda x: " ".join(x))

    return keywords_list


def _create__raw_words__column(directory, source_column, dest_column):

    #
    roots = _extract_keywords_roots_from_database_files(directory).to_list()
    
    #
    files = list(glob.glob(os.path.join(directory, "processed/_*.csv")))
    for file in files:
        data = pd.read_csv(file, encoding="utf-8")
        #
        if source_column not in data.columns:
            continue

        sys.stdout.write(f"--INFO-- Creating `{dest_column}` column in {file}\n")

        text = data[['article', source_column]].copy()
        text = text.dropna()
        
        text[source_column] = text[source_column].str.lower().copy()
        
        
        text['raw_words'] = text[source_column].map(lambda x: TextBlob(x).noun_phrases)
        text = text.explode('raw_words')
        text = text.dropna()
        text['keys'] = text['raw_words'].str.split()
        s = PorterStemmer()
        text['keys'] = text['keys'].map(lambda x: [s.stem(y) for y in x])
        text['keys'] = text['keys'].map(set)
        text['keys'] = text['keys'].map(sorted)
        text['keys'] = text['keys'].map(lambda x: " ".join(x))
        
        text['found'] = text['keys'].map(lambda x: x in roots)
        text = text[text['found'] == True]

        text = text[['article', 'raw_words']]
        text = text.groupby('article', as_index=False).aggregate(lambda x: list(x))
        text['raw_words'] = text['raw_words'].map(set)
        text['raw_words'] = text['raw_words'].map(sorted)
        text['raw_words'] = text['raw_words'].str.join("; ")

        # convert the pandas series to a dictionary
        values_dict = dict(zip(text.article, text.raw_words))
        data[dest_column] = data['article'].map(lambda x: values_dict.get(x, pd.NA))
        #
        data.to_csv(file, sep=",", encoding="utf-8", index=False)


directory = "regtech"
_create__raw_abstract_words__column(directory, source_column='abstract', dest_column='raw_abstract_words')
_create__raw_abstract_words__column(directory, source_column='title', dest_column='raw_title_words')

In [ ]:
import pandas as pd

directory = "regtech"

documents = pd.read_csv(f"{directory}/processed/_documents.csv")

index=2
print(documents.raw_abstract_words[index])
print(documents.raw_title_words[index])

In [ ]:
print(documents.raw_author_keywords[index])
print(documents.raw_index_keywords[index])
print(documents.title[index])

In [ ]:
print(documents.raw_index_keywords[0])